### Time to first protest since George Floyd

George Floyd: 26 May 2020
End of dataset: 20 May 2022

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
protests = pd.read_csv('data/daily_all_locs.csv')

In [3]:
weekly_protests = pd.read_csv('data/combined_final_weekly_data.csv')

# BLM

In [4]:
blm_weekly = weekly_protests[weekly_protests['Actor.Affiliation_x'] != 'Alt-Right']
blm_weekly['EVENT_DATE_x'] = pd.to_datetime(blm_weekly['EVENT_DATE_x'])  
blm_weekly = blm_weekly[(blm_weekly['EVENT_DATE_x'] > '2020-05-25') | (blm_weekly['EVENT_DATE_x'].isna())]
#blm_weekly = blm_weekly.sort_values(by='EVENT_DATE_x')
#blm_weekly = blm_weekly.drop_duplicates(subset='GEOID10_x', keep='first')

Keeping all weeks with 'Protest?' = 0 until first protest occurs (only from week 2020-21 --> 26th May 2020)

In [5]:
weeks = []
for i in range(0,21):
    weeks.append('2020-'+str(i))
    
blm_weekly = blm_weekly[~blm_weekly['Year-Week'].isin(weeks)]

In [6]:
m = (blm_weekly['Protest?_x'].eq(1)
       .groupby(blm_weekly['GEOID10_x'])
       .apply(lambda x: ~x.cummax().shift().fillna(False)))

blm_weekly = blm_weekly[m]

In [7]:
blm_weekly['Not immune'] = blm_weekly.groupby('NAME10_x')['Protest?_x'].transform('max')

Adding number of days since George Floyd for those that do have protest

In [20]:
blm_weekly['Time to first BLM protest since GF'] = blm_weekly['EVENT_DATE_x'] - pd.to_datetime('2020-05-26')
blm_weekly['Time to first BLM protest since GF'] = (blm_weekly['Time to first BLM protest since GF']).dt.days

In [145]:
blm_weekly['Weeks since GF'] = (pd.to_datetime(blm_weekly['Year-Week'] + '-0', format='%Y-%U-%w') - pd.to_datetime('2020-20-0', format='%Y-%U-%w')) / np.timedelta64(1, 'W')

In [146]:
states = blm_weekly.State_x.unique().tolist()

In [147]:
for state in states:
    blm_weekly.loc[blm_weekly['State_x'] == state, state] = 1
    blm_weekly[state] = blm_weekly[state].fillna(0)

In [148]:
blm_weekly.to_csv('data/blm_weekly_first_gf.csv')

### Daily - not currently using this

In [55]:
blm = protests[protests['Actor.Affiliation'] != 'Alt-Right']
blm['EVENT_DATE'] = pd.to_datetime(blm['EVENT_DATE'])  
blm = blm[(blm['EVENT_DATE'] > '2020-05-25') | (blm['EVENT_DATE'].isna())]
blm = blm.sort_values(by='EVENT_DATE')
blm = blm.drop_duplicates(subset='GEOID10', keep='first')
blm['Time to first protest since GF'] = blm['EVENT_DATE'] - pd.to_datetime('2020-05-26')
blm['Time to first protest since GF'] = blm['Time to first protest since GF'].fillna(pd.to_datetime('2022-05-20') - pd.to_datetime('2020-05-26'))                                                              
blm['Time to first protest since GF'] = (blm['Time to first protest since GF']).dt.days

# Right wing

In [77]:
right_weekly = weekly_protests[weekly_protests['Actor.Affiliation_y'] != 'BLM']
right_weekly['EVENT_DATE_y'] = pd.to_datetime(right_weekly['EVENT_DATE_y'])  
right_weekly = right_weekly[(right_weekly['EVENT_DATE_y'] > '2020-05-25') | (right_weekly['EVENT_DATE_y'].isna())]
right_weekly = right_weekly.sort_values(by='EVENT_DATE_y')
right_weekly = right_weekly.drop_duplicates(subset='GEOID10_x', keep='first')

In [79]:
right_weekly['Time to first right protest since GF'] = right_weekly['EVENT_DATE_y'] - pd.to_datetime('2020-05-26')
right_weekly['Time to first right protest since GF'] = right_weekly['Time to first right protest since GF'].fillna(pd.to_datetime('2022-05-20') - pd.to_datetime('2020-05-26'))                                                              
right_weekly['Time to first right protest since GF'] = (right_weekly['Time to first right protest since GF']).dt.days

In [80]:
right_weekly.to_csv('data/right_weekly_first_gf.csv')

# Breonna Taylor

In [5]:
blm_bt = weekly_protests[weekly_protests['Actor.Affiliation_x'] != 'Alt-Right']
blm_bt['EVENT_DATE_x'] = pd.to_datetime(blm_bt['EVENT_DATE_x'])  
blm_bt = blm_bt[(blm_bt['EVENT_DATE_x'] > '2020-03-12') | (blm_bt['EVENT_DATE_x'].isna())]
blm_bt = blm_bt.sort_values(by='EVENT_DATE_x')
blm_bt = blm_bt.drop_duplicates(subset='GEOID10_x', keep='first')

blm_bt['Time to first BLM protest since BT'] = blm_bt['EVENT_DATE_x'] - pd.to_datetime('2020-03-13')
blm_bt['Time to first BLM protest since BT'] = blm_bt['Time to first BLM protest since BT'].fillna(pd.to_datetime('2022-05-20') - pd.to_datetime('2020-03-13'))                                                              
blm_bt['Time to first BLM protest since BT'] = (blm_bt['Time to first BLM protest since BT']).dt.days

blm_bt.to_csv('data/blm_weekly_first_bt.csv')

# Ahmaud Arbery 

In [6]:
blm_aa = weekly_protests[weekly_protests['Actor.Affiliation_x'] != 'Alt-Right']
blm_aa['EVENT_DATE_x'] = pd.to_datetime(blm_aa['EVENT_DATE_x'])  
blm_aa = blm_aa[(blm_aa['EVENT_DATE_x'] > '2020-02-22') | (blm_aa['EVENT_DATE_x'].isna())]
blm_aa = blm_aa.sort_values(by='EVENT_DATE_x')
blm_aa = blm_aa.drop_duplicates(subset='GEOID10_x', keep='first')

blm_aa['Time to first BLM protest since AA'] = blm_aa['EVENT_DATE_x'] - pd.to_datetime('2020-02-23')
blm_aa['Time to first BLM protest since AA'] = blm_aa['Time to first BLM protest since AA'].fillna(pd.to_datetime('2022-05-20') - pd.to_datetime('2020-02-23'))                                                              
blm_aa['Time to first BLM protest since AA'] = (blm_aa['Time to first BLM protest since AA']).dt.days

blm_aa.to_csv('data/blm_weekly_first_aa.csv')